In [1]:
from dotenv import load_dotenv

from flask import Flask, request, render_template
from twilio.twiml.messaging_response import MessagingResponse
import openai
import os
from google.auth import _service_account_info
from google.cloud import storage
import google.auth
from google.cloud import storage
from google.oauth2.service_account import Credentials

In [2]:

load_dotenv()
open_ai_key = os.getenv("OPENAI_API_KEY")
completion = openai.Completion()

start_sequence = "\nAI:"
restart_sequence = "\nHuman: "
# session_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly."

session_prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.\nHuman: Hello, who are you?\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: "

In [3]:
app = Flask(__name__)
openai.api_key = os.getenv("OPENAI_API_KEY")
app.config['SECRET_KEY'] = '323434'

model = "text-davinci-003"


In [4]:
def add_log(response):
    """
    Logs a conversation to a Google Cloud Storage bucket and returns the updated log.

    Args:
        response (str): A string containing a single entry to be added to the log

    Returns:
        str: The updated conversation log
    """
    # Load Google Cloud Storage credentials from environment variables
    credentials = Credentials.from_service_account_file('gpt-pdsbot-svc.json')

    # Connect to the GCS bucket
    client = storage.Client(project='gpt-pdsbot', credentials=credentials)
    bucket = client.bucket("pds_ai_chat_logs")

    # Download the current log from the bucket and convert it to a string
    blob = bucket.get_blob("chat_log.txt")
    chat_log = blob.download_as_string().decode("utf-8")

    # Append the new entry to the log and upload it to the bucket
    new_discussion = response
    chat_log += new_discussion
    blob.upload_from_string(chat_log.encode("utf-8"))

    # Return the updated conversation log
    return chat_log


In [5]:
# I also want to print the most three recent conversations
def print_recent_conversations():
    """
    Prints the most recent 3 conversations from the chat log.
    """
    # Load Google Cloud Storage credentials from environment variables
    credentials = Credentials.from_service_account_file('../gpt-pdsbot-svc.json')

    # Connect to the GCS bucket
    client = storage.Client(project='gpt-pdsbot', credentials=credentials)
    bucket = client.bucket("pds_ai_chat_logs")

    # Download the current log from the bucket and convert it to a string
    blob = bucket.get_blob("chat_log.txt")
    chat_log = blob.download_as_string().decode("utf-8")

    # Split the log into a list of entries
    log_entries = chat_log.split("\n")

    # Print the most recent 3 entries
    print("Most recent conversations:")
    for i in range(1, 4):
        print(log_entries[-i]) 
        
        # Print a separator between entries
        print("-----")
        print("\n")

In [6]:
def ask_chat(question, chat_log=add_log):
    """
    Generates a response to a user's question using the OpenAI API and a specified language model.
    Logs the user's question and the generated response to a chat log.

    Args:
        question (str): The user's input question
        chat_log (function): A function that logs the conversation to a file or database

    Returns:
        str: The AI's generated response to the user's question
    """
    
    # Construct the prompt text by concatenating the chat log, question, and sequence markers
    prompt_text = f'{chat_log}{restart_sequence}{question}{start_sequence}'
    
    # Generate the AI's response using the OpenAI API and the specified model
    response = openai.Completion.create(
        model=model,                # Specify the language model to use
        prompt=prompt_text,         # Provide the starting text prompt for the model
        temperature=0.9,            # Set the level of randomness in the generated text
        max_tokens=150,             # Limit the length of the generated text
        top_p=1,                    # Control the probability threshold for word selection
        frequency_penalty=0,        # Discourage the model from repeating certain words
        presence_penalty=0.6,       # Discourage the model from repeating phrases
        stop=[" Human:", " AI:"]    # Set stopping conditions for the generated text
    )
    
    # Extract the generated text from the OpenAI API response and convert it to a string
    story = str(response['choices'][0]['text'])
    
    # Convert the question and response to strings and log them in a dictionary
    submission = str(question)
    response = str(story)
    sess_dict = {
        "Submission": submission,
        "Response": response
    }       
    
    # Add the dictionary to the chat log and return the generated response
    sess_dict = str(sess_dict)
    chat_log(sess_dict)
    print(sess_dict)
    
    #For this format I prefer visually to not print the story, so I don't return it
    # return story


In [7]:

ask_chat("What do you think is the best Naughty by Nature song?")

{'Submission': 'What do you think is the best Naughty by Nature song?', 'Response': ' Naughty by Nature is most known for the song "O.P.P.," from their 1991 self-titled album. Other popular songs from the group include "Hip Hop Hooray," "Feel Me Flow," and "Everything\'s Gonna Be Alright."'}


In [ ]:
# # Create a web app using Flask
# app = Flask(__name__)



# # Create a route for the home page
# @app.route('/')
# def home():
#     name = 'John'
#     return render_template('home.html', name=name)

    

# # Create a route for the chat page  
# @app.route("/chat")
# def chat():
#     return render_template("chat.html")

# # Create a route for the chatbot    
# @app.route("/get")
# def get_bot_response():
#     userText = request.args.get('msg')
#     return str(ask_chat(userText))

# # Run the web app
# if __name__ == "__main__":
#     app.run()
    

